<h1>James Powell on Core Concepts of Python</h1>

<h2>The protocol view of python</h2>
<p>If you look at the object orientation of python there are three core patterns we have to understand. One of them is protocol view of python. </p>

In [1]:
class Polynomial:
    pass

p1 = Polynomial()
p2 = Polynomial()
p1.coeffs = 1, 2, 3 #x**2 + 2x + 3
p2.coeffs = 3, 4, 3 #x**2 + 4x + 3

<h6>__init__</h6>
<p>Why write in 4 lines when we can write it in two lines? Two lines will do.<p/>

In [2]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs


p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)

print(p1)
print(p2)

<h6>__repr__</h6>
<p>That looks ugly because we are missing the method that corresponds to what happens when we call top level repr function to figure out the representation of our python object.<p/>

In [3]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)


p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)

print(p1)
print(p2)

Polynomial(*(1, 2, 3))
Polynomial(*(3, 4, 3))


<h6>__add__</h6>
<p>What about adding polynomial objects together?<p/>

In [4]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))


p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)

print(p1)
print(p2)

print(p1 + p2)

Polynomial(*(1, 2, 3))
Polynomial(*(3, 4, 3))
Polynomial(*(4, 6, 6))


<h4>What pattern do we see?</h4>
<p>If we have some behaviour we want to implement --> we write some __function__ to implement it. This functions are called dunder methods. </p> 
<p>It has also an abhorrent name <i>data model methods</i>. Python documentation contains a whole list of this methods. Whenever we want to implement some behaviour in python we want to tell python "for this arbitary object do this behaviour, like give the printable representation for this object, perform an addition of this objects."</p>
<p>The pattern is: there is a top-level function or some top-level syntax and there is a corresponding dunder function. The exact arguments that the underscore function takes are determined by python documentation. The default names of the arguments can be picked up by the documentation also. </p>
<p>But there is something more fundamental than this. If we want to implement the summation of two objects x + y --> we implement __add__, if we want to define how we instantiate an object --> we implement __init__. </p>
<h6>__len__</h6>

<p>What a size of a polynomial might me? The highest degree of the polynomial. We will implement __len__ function to tell us the size of the polynomial</p>

In [5]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))
    
    def __len__(self):
        return len(self.coeffs)


p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)

print(len(p1))

3


<h4>The pattern as we see is as follows:</h4>
<p>The python data model is a means by which we can implement protocols. Those protocols have some abstract meaning depending on the object itself. In the case of a polynomial summation means whatever it ment in the math class.</p>
<p>To get its printable representation we use __repr__. Printable representation is typically whatever string we have to type in the console to get a new instance of that object.</p>
<p>In each case that protocol exists, there is some underscore method that implements this protocol, and there is some top-level function like len or top-level syntax like + sign that allows us to invoke that protocol and it all fits together.</p>
<p>When we implement smth like __len__ we do that by delegating back to the protocol itself. __len__ is implemented in terms of len() being called on a constituent object. __add__ is implemented by adding up some components. </p>
<p>If we have some top-level syntax like parentheses that come after the object x(), we call it the __call__ protocol. </p>

In [6]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    def __add__(self, other):
        return Polynomial(*(x + y for x, y in zip(self.coeffs, other.coeffs)))
    
    def __len__(self):
        return len(self.coeffs)
    
    def __call__(self):
        #no idea what should polynomial do
        pass


p1 = Polynomial(1, 2, 3)
p2 = Polynomial(3, 4, 3)


<h2>Metaclasses</h2>
<p>If you know what metaclasses are it is very clear and very obvious for why and when you use it and it is someting you can kind of shelf away in your mind as "oh, this feature does this, this is why I want to use it, i don't have to use it all the time". </p>
<p>Let's imagine there are two teams on the project and one team writes libraty code and the other team writes user code. People on the user side cant touch library code.</p>

In [7]:
# library.py
# there is an actual library.py file in the directory of the project
# the code here is for demonstration purposes

class Base:
    def foo(self):
        return 'foo'

In [ ]:
# user.py
from library import Base

class Derived(Base):
    def bar(self):
        return self.foo()
    
# this code can break if there is no foo method. 

<p><b>What we can do to figure out our code brakes before the runtime?</b> We can write a test for example. We will see that the code fails some time before it hits the runtime production environment.</p> 
<p>But maybe there is one thing that we can add to this code so it fails before it hits the runtime production environment?</p>
<p>For example, we can add assert and see if the Base has no foo method before we even came up to derived class.</p>

In [ ]:
#user.py
from library import Base

assert hasattr(Base, 'foo'), "You broke it!"

class Derived(Base):
    def bar(self):
        return self.foo()

<p>So now we have an early warning that this has broken. And what we are trying to do here is enforce a constraint. The user level enforces a constraint on a library level, in other words, the Derived class is enforcing a constraint on a base class. The Derived class says "the Base class should have this characteristics in order for me to run and be happy. If it does not foo method implemented i will fail."</p>
<br>
<br>
<p>Let's say instead, <b>we are working on a library side and we don't want people to screw code on a user side of the project</b>. And we can't change anything on the user side of the code. And how do we make sure that the code will be implemented on the user side? There are three answers to this. One is metaclasses. But maybe we can try use __build_class__ at first.</p>

In [ ]:
# library3.py
class Base:
    def foo(self):
        return self.bar()

# we write this Base class under the assumption that developer in user department
# will implement bar() method, because if they don't, everything falls apart.

In [ ]:
import sys
sys.modules[__name__].__dict__.clear()
#user.py
from library3 import Base

class Derived(Base):
    def bar(self):
        return foo()

<p>The reason we can call python a protocol oriented language is not just because the python data model is protocol oriented, but the entire python language itself has a notion of hooks and protocols and safety valves within it.</p>
<p>Python code runs from top to bottom linearly and almost every statement except the two of them are actually executable rutime code. In C++ or Java a class statement is not an executable code. In python it is.</p>

In [8]:
import sys
sys.modules[__name__].__dict__.clear()

def _():
    class Base:
        pass

# dis stands for disassemble
from dis import dis
dis(_)

  5           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Base at 0x7fd7707bfb70, file "<ipython-input-8-98bf1b7926fc>", line 5>)
              4 LOAD_CONST               2 ('Base')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Base')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Base)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


<p><b>LOAD_BUILD_CLASS</b> is actual executable runtime instruction in the python enterpreteur for creating a class. There typycally in python tends to be a correspondence between some top-level syntax or function and some underscore method that implements that function. There happens to be some top-level mechanism here for building a class (it is not explicitly a function). It turns out in python there is a hook, there is an underscore function that allows us to do things with the process of building classes.</p>

In [ ]:
import sys
sys.modules[__name__].__dict__.clear()

# library4.py
# class Base:
#     def foo(self):
#         return 'bar'

# old_bc = __build_class__

# def my_bc(*a, **kw):
#     print('my buildclass -->', a, kw)
#     return old_bc(*a, **kw)


# import builtins
# builtins.__build_class__ = my_bc

In [1]:
import sys
sys.modules[__name__].__dict__.clear()

#user.py
from library4 import Base

class Derived(Base):
    def bar(self):
        return 'bar'

my buildclass --> (<function Derived at 0x7f9250a83d08>, 'Derived', <class 'library4.Base'>) {}


<p>So we can actually catch the building of the class! We passed a function, the name of the class <i>'Derived'</i>, the bases <i>class 'library4.Base'</i> and <i>function Derived at 0x7f50f86b2ea0</i>.</p>
<p>So we can add our assert into the building class. </p>

In [1]:
import sys
sys.modules[__name__].__dict__.clear()

class Base:
    def foo(self):
        return 'bar'

old_bc = __build_class__
# the base is optional argument because not everything has to have a baseclass
def my_bc(fun, name, base=None, **kw):
    if base is Base:
        print('check if bar method is defined')
        return old_bc(fun, name, **kw)
    if base is not None:
        return old_bc(fun, name, base, **kw)
    

# import builtins
# builtins.__build_class__ = my_bc

In [1]:
import sys
sys.modules[__name__].__dict__.clear()

#user.py
from library5 import Base

class Derived(Base):
    def bar(self):
        return 'bar'

check if bar method is defined


<p>This is not typically what we do. It is to show us that idea of python being a protocol-oriented language is actually quite a fundamental piece of python. Almost everything that a python language does in an execution context like building classed, creating functions, importing modules, you can find the hook to get into that and start doing things that we want to do.</p>
<p>__build_class__ is available for us to use but that is not how we solve the this problem. There are two fundamental features of python that people use to enforce constraints from derived classes to base classes. The first one is the metaclass.</p>
<p><b>Metaclasses</b> are merely classes that derive from type that have some special methods on them (we have to look into the documentation to find out what this methods are) that <b>allow us to intercept the construction of derived types</b>.</p>

In [ ]:
# library6.py
import sys
sys.modules[__name__].__dict__.clear()

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if not 'bar' in body:
            raise TypeError("bad user class")
        print('BaseMeta.__new__', cls, name, bases, body)
        return super().__new__(cls, name, bases, body)
    
class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()

In [1]:
import sys
sys.modules[__name__].__dict__.clear()

from library6 import Base

class Derived(Base):
    def bar(self):
        return 'bar'

BaseMeta.__new__ <class 'library6.BaseMeta'> Base () {'__module__': 'library6', '__qualname__': 'Base', 'foo': <function Base.foo at 0x7f9814b73d90>}
BaseMeta.__new__ <class 'library6.BaseMeta'> Derived (<class 'library6.Base'>,) {'__module__': 'builtins', '__qualname__': 'Derived', 'bar': <function Derived.bar at 0x7f9814b737b8>}


<p>We can see that it got called with our Derived class. The last argument <i>{'__module__': 'builtins', '__qualname__': 'Derived', 'bar': function Derived.bar at 0x7f4941758ae8}</i> is the dictionary with all the methods of that class.</p>
<p>So in order to enforce constraints on the derived class we have to find ways to intercept the construction of the classes. Metaclassed, despite being a "very complicated feature" is a tool for enforcing constraints down the class hierarchy from a base class to the derived class. </p>

<p>The third approach is a variation of the metaclass approach. <b>In python 3.6 new feature was introduced called __init_subclass__ that gives us a method of hooking into creating of a subclass.</b></p>

In [5]:
# library7.py
class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if name != 'Base' and not 'bar' in body:
            raise TypeError("bad user class")
        print('BaseMeta.__new__', cls, name, bases, body)
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()
    def __init_subclass__(self, *a, **kw):
        print('init_subclass', a, kw)
        return super().__init_subclass__(*a, **kw)

# user.py
class Derived(Base):
    def bar(self):
        return 'bar'
        


BaseMeta.__new__ <class 'BaseMeta'> Base () {'__module__': 'builtins', '__qualname__': 'Base', 'foo': <function Base.foo at 0x7f98142fbf28>, '__init_subclass__': <function Base.__init_subclass__ at 0x7f98142fbd90>, '__classcell__': <cell at 0x7f9814b806a8: empty>}
BaseMeta.__new__ <class 'BaseMeta'> Derived (<class 'Base'>,) {'__module__': 'builtins', '__qualname__': 'Derived', 'bar': <function Derived.bar at 0x7f9814312598>}
init_subclass () {}


<h2>Decorators</h2>
<p>Python is a live language. And function definition in python is actually a live thing.</p>

In [23]:
def add(x, y=10):
    return x + y

print(add)

<function add at 0x7f9814312158>


<p>Python interpreter tells where actually in memory this function exists. And this function is an object, so we can ask all sorts of things like what is name of it.</p>

In [24]:
print(add.__name__)

add


In [25]:
print(add.__module__)

None


In [26]:
print(add.__defaults__)

(10,)


In [27]:
# ouputs the actual byte code of our add function
print(add.__code__.co_code)

b'|\x00|\x01\x17\x00S\x00'


In [28]:
# ouputs how many local variables does it have
print(add.__code__.co_nlocals)

2


In [29]:
# outputs what are the variable names in this function
print(add.__code__.co_varnames)

('x', 'y')


<p>Every python structure that you interact with (whether it is an object or a function or a generator) has some runtime life. We can see in in memory and ask questions. </p>

In [30]:
#inspect function can even give us a source code
from inspect import getsource
print(getsource(add))

#and we there are many more useful functions

def add(x, y=10):
    return x + y



In [33]:
from time import time

def add (x, y=10):
    return x + y

before = time()
print('add(10):', add(10))
after = time()
print('time_take:', after - before)

add(10): 20
time_take: 0.00018215179443359375


In [36]:
from time import time

def timer(func):
    def f(*args, **kwargs):
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print('elapsed', after - before)
        return rv
    return f

@timer
def add(x, y=10):
    return x + y

print(add(20))

elapsed 2.1457672119140625e-06
30


<p>A decorator is merely a syntax that is equivalent to the syntax <i>sub = timer(sub).</i></p> 
<p>Its syntax fits into the ability to dynamically construct a function to wrap this behaviour. So we were able to slip in the extra functionality we want withour rewriting the code. With *args and **kwargs our decorator works on functions with any parameters.</p>

In [46]:
def ntimes(n):
    def inner(f):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                print('running {.__name__}'.format(f))
                rv = f(*args, **kwargs)
            return rv
        return wrapper
    return inner

@ntimes(2)
def add(x, y=10):
    return x + y

print(add(5))

running add
running add
15


<p>This is called <b>higher-order decorators</b>. It is a fairly simplistic extension of the idea "a function can return a function". There is a concept "closure object duality".</p>